In [ ]:
from typing import Callable, Dict, List, Optional
import uuid
import time

In [ ]:
class PromptUnit:

    def __init__(
            self,
            role: str,
            content: str,
            status: Optional[str] = "active",
            metadata: Optional[Dict] = None,
            id: Optional[str] = None,    
        ):
        self.role = role # user, assistant, system, tool, tool_response
        self.content = content # can be text, image, audio, video, file, etc.
        self.status = status # locked, hidden, readonly, etc.
        self.metadata = metadata or {} # 附加边界控制信息
        self.id = str(uuid.uuid4()) if id is None else id # 唯一标识符
    
    def is_visible(self) -> bool:
        return self.status not in ['hidden', 'deleted']

    def is_editable(self) -> bool:
        return self.status not in ['locked', 'readonly', 'deleted']
    
    def to_dict(self) -> Dict:
        return {
            'role': self.role,
            'content': self.content,
            'status': self.status,
            'metadata': self.metadata,
            'id': self.id,
        }

In [ ]:
# 状态变更事件
class StateChangeEvent:
    
    def __init__(
        self,
        context_id: str,
        prompt_unit_id: str,
        old_status: Optional[str],
        new_status: Optional[str],
        timestamp: Optional[float] = None,
    ):
        self.context_id = context_id
        self.prompt_unit_id = prompt_unit_id
        self.old_status = old_status
        self.new_status = new_status
        self.timestamp = timestamp or time.time()

In [ ]:
# 发布-订阅机制
class StateChangeNotifier:
    
    def __init__(self):
        self.subscribers: Dict[str, List[Callable[[StateChangeEvent], None]]] = {}
    
    def subscribe(self, event_type: str, callback: Callable[[StateChangeEvent], None]):
        if event_type not in self.subscribers:
            self.subscribers[event_type] = []
        self.subscribers[event_type].append(callback)
    
    def notify(self, event_type: str, event: StateChangeEvent):
        for callback in self.subscribers.get(event_type, []):
            callback(event)

In [ ]:
# 示例用法
notifier = StateChangeNotifier()

def print_handler(event: StateChangeEvent):
    print(f"Context {event.context_id} PromptUnit {event.prompt_unit_id} changed from {event.old_status} to {event.new_status} at {event.timestamp}")

def database_handler(event: StateChangeEvent):
    # 这里可以添加代码将事件记录到数据库
    print(f"Database updated for Context {event.context_id} PromptUnit {event.prompt_unit_id}")

notifier.subscribe("status_change", print_handler)
notifier.subscribe("status_change", database_handler)

def update_prompt_unit_status(context_id: str, prompt_unit: PromptUnit, new_status: str):
    old_status = prompt_unit.status
    prompt_unit.status = new_status
    event = StateChangeEvent(
        context_id=context_id,
        prompt_unit_id=prompt_unit.id,
        old_status=old_status,
        new_status=new_status
    )
    notifier.notify("status_change", event)


# 示例流程
prompt_unit = PromptUnit(role="user", content="Hello, how are you?", status="active")
update_prompt_unit_status(context_id="context1", prompt_unit=prompt_unit, new_status="interrupted")